# Data Loading, Storage, and File Formats

## Description

### GitHub連結

### Required Datasets

## Required Packages

In [1]:
import numpy as np
import pandas as pd

import sys
import csv
import json
from lxml import objectify
from io import StringIO
import requests
import sqlite3
import sqlalchemy as sqla

## 6.1 Reading and Writing Data in Text Format

##### Parsing functions in pandas

| Function | Description |
| - | - |
| read_csv | Load delimited data from a file, URL, or file-like object; use comma as default delimiter |
| read_table | Load delimited data from a file, URL, or file-like object; use tab ('\t') as default delimiter |
| read_fwf | Read data in fixed-width column format (i.e., no delimiters) |
| read_clipboard | Version of read_table that reads data from the clipboard; useful for converting tables from web pages |
| read_excel | Read tabular data from an Excel XLS or XLSX file |
| read_hdf | Read HDF5 files written by pandas |
| read_html | Read all tables found in the given HTML document |
| read_json | Read data from a JSON (JavaScript Object Notation) string representation |
| read_msgpack | Read pandas data encoded using the MessagePack binary format |
| read_pickle | Read an arbitrary object stored in Python pickle format |
| read_sas | Read a SAS dataset stored in one of the SAS system’s custom storage formats |
| read_sql | Read the results of a SQL query (using SQLAlchemy) as a pandas DataFrame |
| read_stata | Read a dataset from Stata file format |
| read_feather | Read the Feather binary file format |

In [2]:
!cat src/csv/ex1.csv

a,b,c,d,message
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

In [3]:
df = pd.read_csv('src/csv/ex1.csv')
df

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [4]:
pd.read_table('src/csv/ex1.csv', sep=',')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [5]:
!cat src/csv/ex2.csv

1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

In [6]:
pd.read_csv('src/csv/ex2.csv', header=None)

,0,1,2,3,4
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [7]:
pd.read_csv('src/csv/ex2.csv', names=['a', 'b', 'c', 'd', 'message'])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [8]:
names = ['a', 'b', 'c', 'd', 'message']

In [9]:
pd.read_csv('src/csv/ex2.csv', names=names, index_col='message')

,a,b,c,d
message,,,,
hello,1,2,3,4
world,5,6,7,8
foo,9,10,11,12


In [10]:
!cat src/csv/csv_mindex.csv

key1,key2,value1,value2
one,a,1,2
one,b,3,4
one,c,5,6
one,d,7,8
two,a,9,10
two,b,11,12
two,c,13,14
two,d,15,16


In [11]:
parsed = pd.read_csv('src/csv/csv_mindex.csv', index_col=['key1', 'key2'])
parsed

value1  value2
key1 key2                
one  a          1       2
     b          3       4
     c          5       6
     d          7       8
two  a          9      10
     b         11      12
     c         13      14
     d         15      16

In [12]:
list(open('src/txt/ex3.txt'))

['            A         B         C\n',
 'aaa -0.264438 -1.026059 -0.619500\n',
 'bbb  0.927272  0.302904 -0.032399\n',
 'ccc -0.264273 -0.386314 -0.217601\n',
 'ddd -0.871858 -0.348382  1.100491\n']

In [13]:
result = pd.read_table('src/txt/ex3.txt', sep='\s+')
result

,A,B,C
aaa,-0.264438,-1.026059,-0.619500
bbb,0.927272,0.302904,-0.032399
ccc,-0.264273,-0.386314,-0.217601
ddd,-0.871858,-0.348382,1.100491


In [14]:
!cat src/csv/ex4.csv

# hey!
a,b,c,d,message
# just wanted to make things more difficult for you
# who reads CSV files with computers, anyway?
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

In [15]:
pd.read_csv('src/csv/ex4.csv', skiprows=[0, 2, 3])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [16]:
!cat src/csv/ex5.csv

something,a,b,c,d,message
one,1,2,3,4,NA
two,5,6,,8,world
three,9,10,11,12,foo

In [17]:
result = pd.read_csv('src/csv/ex5.csv')
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [18]:
pd.isnull(result)

,something,a,b,c,d,message
0,False,False,False,False,False,True
1,False,False,False,True,False,False
2,False,False,False,False,False,False


In [19]:
result = pd.read_csv('src/csv/ex5.csv', na_values=['NULL'])
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [20]:
sentinels = {'message': ['foo', 'NA'], 'something': ['two']}
pd.read_csv('src/csv/ex5.csv', na_values=sentinels)

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,NaN,5,6,NaN,8,world
2,three,9,10,11.0,12,NaN


##### Some read_csv/read_table function arguments

| Argument | Description |
| - | - |
| path | String indicating filesystem location, URL, or file-like object |
| sep or delimiter | Character sequence or regular expression to use to split fields in each row |
| header | Row number to use as column names; defaults to 0 (first row), but should be None if there is no header row |
| index_col | Column numbers or names to use as the row index in the result; can be a single name/number or a list of them for a hierarchical index |
| names | List of column names for result, combine with header=None |
| skiprows | Number of rows at beginning of file to ignore or list of row numbers (starting from 0) to skip. |
| na_values | Sequence of values to replace with NA. |
| comment | Character(s) to split comments off the end of lines. |
| parse_dates | Attempt to parse data to datetime; False by default. If True, will attempt to parse all columns. Otherwise can specify a list of column numbers or name to parse. If element of list is tuple or list, will combine multiple columns together and parse to date (e.g., if date/time split across two columns). |
| keep_date_col | If joining columns to parse date, keep the joined columns;Falseby default. |
| converters | Dict containing column number of name mapping to functions (e.g., {'foo': f} would apply the functionfto all values in the'foo'column). |
| dayfirst | When parsing potentially ambiguous dates, treat as international format (e.g., 7/6/2012 -> June 7, 2012);Falseby default. |
| date_parser | Function to use to parse dates. |
| nrows | Number of rows to read from beginning of file. |
| iterator | Return a TextParser object for reading file piecemeal. |
| chunksize | For iteration, size of file chunks. |
| skip_footer | Number of lines to ignore at end of file. |
| verbose | Print various parser output information, like the number of missing values placed in non-numeric columns. |
| encoding | Text encoding for Unicode (e.g., 'utf-8' for UTF-8 encoded text). |
| squeeze | If the parsed data only contains one column, return a Series. |
| thousands | Separator for thousands (e.g., ',' or '.'). |

### Reading Text Files in Pieces

In [21]:
pd.options.display.max_rows = 10

In [22]:
result = pd.read_csv('src/csv/ex6.csv')
result

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q
...,...,...,...,...,...
9995,2.311896,-0.417070,-1.409599,-0.515821,L
9996,-0.479893,-0.650419,0.745152,-0.646038,E
9997,0.523331,0.787112,0.486066,1.093156,K
9998,-0.362559,0.598894,-1.843201,0.887292,G


In [23]:
pd.read_csv('src/csv/ex6.csv', nrows=5)

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q


In [24]:
chunker = pd.read_csv('src/csv/ex6.csv', chunksize=1000)
chunker

In [25]:
chunker = pd.read_csv('src/csv/ex6.csv', chunksize=1000)

In [26]:
tot = pd.Series([])

/var/folders/_5/rbp8ycrj4t95c58km4gtz5340000gn/T/ipykernel_8163/1115746282.py:1: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  tot = pd.Series([])


In [27]:
for piece in chunker:
    tot = tot.add(piece['key'].value_counts(), fill_value=0)

In [28]:
chunker = pd.read_csv('src/csv/ex6.csv', chunksize=1000)

In [29]:
tot[:10]

0    151.0
1    146.0
2    152.0
3    162.0
4    171.0
5    157.0
6    166.0
7    164.0
8    162.0
9    150.0
dtype: float64

### Writing Data to Text Format

In [30]:
data = pd.read_csv('src/csv/ex5.csv')
data

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [31]:
data.to_csv('src/csv/out.csv')

In [32]:
!cat src/csv/out.csv

,something,a,b,c,d,message
0,one,1,2,3.0,4,
1,two,5,6,,8,world
2,three,9,10,11.0,12,foo


In [33]:
data.to_csv(sys.stdout, sep='|')

|something|a|b|c|d|message
0|one|1|2|3.0|4|
1|two|5|6||8|world
2|three|9|10|11.0|12|foo


In [34]:
data.to_csv(sys.stdout, na_rep='NULL')

,something,a,b,c,d,message
0,one,1,2,3.0,4,NULL
1,two,5,6,NULL,8,world
2,three,9,10,11.0,12,foo


In [35]:
data.to_csv(sys.stdout, index=False, header=False)

one,1,2,3.0,4,
two,5,6,,8,world
three,9,10,11.0,12,foo


In [36]:
data.to_csv(sys.stdout, index=False, columns=['a', 'b', 'c'])

a,b,c
1,2,3.0
5,6,
9,10,11.0


In [37]:
dates = pd.date_range('1/1/2000', periods=7)

In [38]:
ts = pd.Series(np.arange(7), index=dates)

In [39]:
ts.to_csv('src/csv/tseries.csv')

In [40]:
!cat src/csv/tseries.csv

,0
2000-01-01,0
2000-01-02,1
2000-01-03,2
2000-01-04,3
2000-01-05,4
2000-01-06,5
2000-01-07,6


### Working with Delimited Formats

In [41]:
!cat src/csv/ex7.csv

"a","b","c"
"1","2","3"
"1","2","3"


In [42]:
f = open('src/csv/ex7.csv')

In [43]:
reader = csv.reader(f)

In [44]:
for line in reader:
    print(line)

['a', 'b', 'c']
['1', '2', '3']
['1', '2', '3']


In [45]:
with open('src/csv/ex7.csv') as f:
    lines = list(csv.reader(f))

In [46]:
header, values = lines[0], lines[1:]

In [47]:
data_dict = {h: v for h, v in zip(header, zip(*values))}
data_dict

{'a': ('1', '1'), 'b': ('2', '2'), 'c': ('3', '3')}

In [48]:
class my_dialect(csv.Dialect):
    lineterminator = '\n'
    delimiter = ';'
    quotechar = '"'
    quoting = csv.QUOTE_MINIMAL

In [49]:
# reader = csv.reader(f, dialect=my_dialect)

In [50]:
reader = csv.reader('f', delimiter='|')

| Argument | Description |
| - | - |
| delimiter | One-character string to separate fields; defaults to ','. |
| lineterminator | Line terminator for writing; defaults to '\r\n'. Reader ignores this and recognizes cross-platform line terminators. |
| quotechar | Quote character for fields with special characters (like a delimiter); default is '"'. |
| quoting | Quoting convention. Options includecsv.QUOTE_ALL(quote all fields),csv.QUOTE_MINI MAL(only fields with special characters like the delimiter),csv.QUOTE_NONNUMERIC, and csv.QUOTE_NONE (no quoting). See Python’s documentation for full details. Defaults to QUOTE_MINIMAL. |
| skipinitialspace | Ignore whitespace after each delimiter; default is False. |
| doublequote | How to handle quoting character inside a field; if True, it is doubled (see online documentation for full detail and behavior). |
| escapechar | String to escape the delimiter ifquotingis set tocsv.QUOTE_NONE; disabled by default. |

```python
with open('mydata.csv', 'w') as f:
    writer = csv.writer(f, dialect=my_dialect)
    writer.writerow(('one', 'two', 'three'))
    writer.writerow(('1', '2', '3'))
    writer.writerow(('4', '5', '6'))
    writer.writerow(('7', '8', '9'))
```

### JSON Data

In [51]:
obj = """
    {"name": "Wes",
     "places_lived": ["United States", "Spain", "Germany"],
     "pet": null,
     "siblings": [{"name": "Scott", "age": 30, "pets": ["Zeus", "Zuko"]},
                  {"name": "Katie", "age": 38,
                   "pets": ["Sixes", "Stache", "Cisco"]}]
    }
"""

In [52]:
result = json.loads(obj)
result

{'name': 'Wes',
 'places_lived': ['United States', 'Spain', 'Germany'],
 'pet': None,
 'siblings': [{'name': 'Scott', 'age': 30, 'pets': ['Zeus', 'Zuko']},
  {'name': 'Katie', 'age': 38, 'pets': ['Sixes', 'Stache', 'Cisco']}]}

In [53]:
asjson = json.dumps(result)

In [54]:
siblings = pd.DataFrame(result['siblings'], columns=['name', 'age'])
siblings

,name,age
0,Scott,30
1,Katie,38


In [55]:
!cat src/json/example.json

[{"a": 1, "b": 2, "c": 3},
 {"a": 4, "b": 5, "c": 6},
 {"a": 7, "b": 8, "c": 9}]


In [56]:
data = pd.read_json('src/json/example.json')
data

,a,b,c
0,1,2,3
1,4,5,6
2,7,8,9


In [57]:
print(data.to_json())

{"a":{"0":1,"1":4,"2":7},"b":{"0":2,"1":5,"2":8},"c":{"0":3,"1":6,"2":9}}


In [58]:
print(data.to_json(orient='records'))

[{"a":1,"b":2,"c":3},{"a":4,"b":5,"c":6},{"a":7,"b":8,"c":9}]


### XML and HTML: Web Scraping

In [59]:
tables = pd.read_html('src/html/fdic_failed_bank_list.html')

In [60]:
len(tables)

1

In [61]:
failures = tables[0]
failures.head()

,Bank Name,City,ST,CERT,Acquiring Institution,Closing Date,Updated Date
0,Allied Bank,Mulberry,AR,91,Today's Bank,"September 23, 2016","November 17, 2016"
1,The Woodbury Banking Company,Woodbury,GA,11297,United Bank,"August 19, 2016","November 17, 2016"
2,First CornerStone Bank,King of Prussia,PA,35312,First-Citizens Bank & Trust Company,"May 6, 2016","September 6, 2016"
3,Trust Company Bank,Memphis,TN,9956,The Bank of Fayette County,"April 29, 2016","September 6, 2016"
4,North Milwaukee State Bank,Milwaukee,WI,20364,First-Citizens Bank & Trust Company,"March 11, 2016","June 16, 2016"


In [62]:
close_timestamps = pd.to_datetime(failures['Closing Date'])

In [63]:
close_timestamps.dt.year.value_counts()

2010    157
2009    140
2011     92
2012     51
2008     25
       ... 
2004      4
2001      4
2007      3
2003      3
2000      2
Name: Closing Date, Length: 15, dtype: int64

#### Parsing XML with lxml.objectify

```xml
<INDICATOR>
    <INDICATOR_SEQ>373889</INDICATOR_SEQ>
    <PARENT_SEQ></PARENT_SEQ>
    <AGENCY_NAME>Metro-North Railroad</AGENCY_NAME>
    <INDICATOR_NAME>Escalator Availability</INDICATOR_NAME>
    <DESCRIPTION>Percent of the time that escalators are operational systemwide. The availability rate is based on physical observations performed the morning of regular business days only. This is a new indicator the agency began reporting in 2009.</DESCRIPTION>
    <PERIOD_YEAR>2011</PERIOD_YEAR>
    <PERIOD_MONTH>12</PERIOD_MONTH>
    <CATEGORY>Service Indicators</CATEGORY>
    <FREQUENCY>M</FREQUENCY>
    <DESIRED_CHANGE>U</DESIRED_CHANGE>
    <INDICATOR_UNIT>%</INDICATOR_UNIT>
    <DECIMAL_PLACES>1</DECIMAL_PLACES>
    <YTD_TARGET>97.00</YTD_TARGET>
    <YTD_ACTUAL></YTD_ACTUAL>
    <MONTHLY_TARGET>97.00</MONTHLY_TARGET>
    <MONTHLY_ACTUAL></MONTHLY_ACTUAL>
</INDICATOR>
```

In [64]:
path = 'src/xml/Performance_MNR.xml'
parsed = objectify.parse(open(path))
root = parsed.getroot()

In [65]:
data = []
skip_fields = ['PARENT_SEQ', 'INDICATOR_SEQ',
               'DESIRED_CHANGE', 'DECIMAL_PLACES']

In [66]:
for elt in root.INDICATOR:
    el_data = {}
    for child in elt.getchildren():
        if child.tag in skip_fields:
            continue
        el_data[child.tag] = child.pyval
    data.append(el_data)

In [67]:
perf = pd.DataFrame(data)

In [68]:
perf.head()

,AGENCY_NAME,INDICATOR_NAME,DESCRIPTION,PERIOD_YEAR,PERIOD_MONTH,CATEGORY,FREQUENCY,INDICATOR_UNIT,YTD_TARGET,YTD_ACTUAL,MONTHLY_TARGET,MONTHLY_ACTUAL
0,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,1,Service Indicators,M,%,95.0,96.9,95.0,96.9
1,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,2,Service Indicators,M,%,95.0,96.0,95.0,95.0
2,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,3,Service Indicators,M,%,95.0,96.3,95.0,96.9
3,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,4,Service Indicators,M,%,95.0,96.8,95.0,98.3
4,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,5,Service Indicators,M,%,95.0,96.6,95.0,95.8


In [69]:
tag = '<a href="http://www.google.com">Google</a>'
root = objectify.parse(StringIO(tag)).getroot()

In [70]:
root

<Element a at 0x1101eef40>

In [71]:
root.get('href')

'http://www.google.com'

In [72]:
root.text

'Google'

## 6.2 Binary Data Formats

In [73]:
frame = pd.read_csv('src/csv/ex1.csv')

In [74]:
frame

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [75]:
frame.to_pickle('src/pickle/frame_pickle')

In [76]:
pd.read_pickle('src/pickle/frame_pickle')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


### Using HDF5 Format

In [77]:
frame = pd.DataFrame({'a': np.random.randn(100)})

In [78]:
store = pd.HDFStore('src/hdf/mydata.h5')

In [79]:
store['obj1'] = frame

In [80]:
store['obj1_col'] = frame['a']

In [81]:
store

<class 'pandas.io.pytables.HDFStore'>
File path: src/hdf/mydata.h5

In [82]:
store['obj1']

,a
0,2.144339
1,0.371705
2,-0.572948
3,0.948129
4,0.400224
...,...
95,-0.384520
96,-0.338431
97,-1.163588
98,1.056140


In [83]:
store.put('obj2', frame, format='table')

In [84]:
store.select('obj2', where=['index >= 10 and index <= 15'])

,a
10,-0.729572
11,0.011893
12,1.536946
13,-0.624776
14,0.692309
15,-0.042415


In [85]:
store.close()

In [86]:
frame.to_hdf('mydata.h5', 'obj3', format='table')

In [87]:
pd.read_hdf('mydata.h5', 'obj3', where=['index < 5'])

,a
0,2.144339
1,0.371705
2,-0.572948
3,0.948129
4,0.400224


### Reading Microsoft Excel Files

In [88]:
xlsx = pd.ExcelFile('src/xlsx/ex1.xlsx')

In [89]:
pd.read_excel(xlsx, 'Sheet1')

,Unnamed: 0,a,b,c,d,message
0,0,1,2,3,4,hello
1,1,5,6,7,8,world
2,2,9,10,11,12,foo


In [90]:
frame = pd.read_excel('src/xlsx/ex1.xlsx', 'Sheet1')
frame

,Unnamed: 0,a,b,c,d,message
0,0,1,2,3,4,hello
1,1,5,6,7,8,world
2,2,9,10,11,12,foo


In [91]:
writer = pd.ExcelWriter('src/xlsx/ex2.xlsx')

In [92]:
frame.to_excel(writer, 'Sheet1')

In [93]:
writer.save()

In [94]:
frame.to_excel('src/xlsx/ex2.xlsx')

## 6.3 Interacting with Web APIs

In [95]:
url = 'https://api.github.com/repos/pandas-dev/pandas/issues'

In [96]:
resp = requests.get(url)
resp

<Response [200]>

In [97]:
data = resp.json()

In [98]:
data[0]['title']

"TST: Don't use autouse fixture in test_eval"

In [99]:
issues = pd.DataFrame(data, columns=['number', 'title',
                                     'labels', 'state'])
issues

,number,title,labels,state
0,45832,TST: Don't use autouse fixture in test_eval,"[{'id': 127685, 'node_id': 'MDU6TGFiZWwxMjc2OD...",open
1,45831,TST: Don't use autouse fixture in test_stata,"[{'id': 127685, 'node_id': 'MDU6TGFiZWwxMjc2OD...",open
2,45830,DEPS: unpin numpydoc #39688,"[{'id': 134699, 'node_id': 'MDU6TGFiZWwxMzQ2OT...",open
3,45825,BUG: infer_dtype fails on PeriodIndex,"[{'id': 76811, 'node_id': 'MDU6TGFiZWw3NjgxMQ=...",open
4,45824,BUG: concat doesn't preserve attrs,"[{'id': 76811, 'node_id': 'MDU6TGFiZWw3NjgxMQ=...",open
...,...,...,...,...
25,45790,BUG: `df.to_hdf` fails when `DatetimeIndex` ha...,"[{'id': 76811, 'node_id': 'MDU6TGFiZWw3NjgxMQ=...",open
26,45789,BUG: Date time slicing fails for `freq=pd.Date...,"[{'id': 76811, 'node_id': 'MDU6TGFiZWw3NjgxMQ=...",open
27,45787,BUG: loc raising uncontrolled error when more ...,"[{'id': 2822098, 'node_id': 'MDU6TGFiZWwyODIyM...",open
28,45785,Regression: Loc raising when indexing mi with ...,"[{'id': 2822098, 'node_id': 'MDU6TGFiZWwyODIyM...",open


## 6.4 Interacting with Databases

In [100]:
query = """
CREATE TABLE test
(a VARCHAR(20), b VARCHAR(20),
c REAL,        d INTEGER
);
"""

In [101]:
con = sqlite3.connect('mydata.sqlite')

In [102]:
con.execute(query)

OperationalError: table test already exists

In [ ]:
con.commit()

In [ ]:
data = [('Atlanta', 'Georgia', 1.25, 6),
        ('Tallahassee', 'Florida', 2.6, 3),
        ('Sacramento', 'California', 1.7, 5)]

In [ ]:
stmt = "INSERT INTO test VALUES(?, ?, ?, ?)"

In [ ]:
con.executemany(stmt, data)

In [ ]:
con.commit()

In [ ]:
cursor = con.execute('select * from test')

In [ ]:
rows = cursor.fetchall()
rows

[('Atlanta', 'Georgia', 1.25, 6),
 ('Tallahassee', 'Florida', 2.6, 3),
 ('Sacramento', 'California', 1.7, 5)]

In [ ]:
cursor.description

(('a', None, None, None, None, None, None),
 ('b', None, None, None, None, None, None),
 ('c', None, None, None, None, None, None),
 ('d', None, None, None, None, None, None))

In [ ]:
pd.DataFrame(rows, columns=[x[0] for x in cursor.description])

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,3
2,Sacramento,California,1.70,5


In [ ]:
db = sqla.create_engine('sqlite:///mydata.sqlite')

In [ ]:
pd.read_sql('select * from test', db)

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,3
2,Sacramento,California,1.70,5


## 6.5 Conclusion